In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus, Mars 
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2025-05-24", scale="utc").tdb
date_flyby_Earth_1 = Time("2026-11-25", scale="utc").tdb
date_flyby_Venus = Time("2027-03-10", scale="utc").tdb
date_flyby_Earth_2 = Time("2028-11-07", scale="utc").tdb
date_flyby_Mars = Time("2029-11-25", scale="utc").tdb
date_arrival = Time("2034-03-29", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth_1))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
earth_3 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth_2))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth_1)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_earth_3 = Orbit.from_ephem(Sun, earth_3, date_flyby_Earth_2)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_earth_2)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_earth_2, ss_ven)
ss_trans_2, ss_target_2 = ss_earth_2.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_ven, ss_earth_3)
ss_trans_3, ss_target_3 = ss_ven.apply_maneuver(man_lambert_3, intermediate=True)

man_lambert_4 = Maneuver.lambert(ss_earth_3, ss_mar)
ss_trans_4, ss_target_4 = ss_earth_3.apply_maneuver(man_lambert_4, intermediate=True)

man_lambert_5 = Maneuver.lambert(ss_mar, ss_jup)
ss_trans_5, ss_target_5 = ss_mar.apply_maneuver(man_lambert_5, intermediate=True)

In [7]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(earth_2, date_flyby_Earth_1, label="Earth_2")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_3, date_flyby_Earth_2, label="Earth_3")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=360 * u.deg), color="black", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=280 * u.deg), color="blue", label="Transfer orbit V-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=255 * u.deg), color="black", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_4.sample(max_anomaly=161 * u.deg), color="black", label="Transfer orbit M-J"
)

plotter.plot_trajectory(
    ss_trans_5.sample(max_anomaly=161 * u.deg), color="black", label="Transfer orbit M-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [10]:
nh_tof_1 = date_flyby_Earth_1 - date_launch # время полета Земля1 - Венера

r_earth_1, v_earth_1 = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth_1).rv(date_flyby_Earth_1)


(v0_Ea1, vk_Ven), = iod.lambert(Sun.k, r_earth_1, r_earth_2, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=181.9999999665479>

In [12]:
# Гиперболический избыток скорости при отлете от Земли
V_inf = (norm(v0_Ea1 - v_earth_1)).to(u.km / u.s)
V_inf

<Quantity 0.12263323 km / s>

In [13]:
# Гиперболический избыток скорости при подлете к Земле_1
V_inf_end = (norm(vk_Ven - v_earth_2)).to(u.km / u.s)
V_inf_end

<Quantity 0.09451359 km / s>

In [17]:
nh_tof_2 = date_flyby_Venus - date_flyby_Earth_1 # время полета Земля_1 - Венера

(v0_Ven, vk_Ea2), = iod.lambert(Sun.k, r_earth_2, r_ven, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=443.00000002187403>

In [9]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 14.79794094 km / s>

In [10]:
nh_tof_3 = date_arrival - date_flyby_Earth # время полета Земля2 - Юпитер

(V0_Ea2, Vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=769.9999999984603>

In [11]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 8.15931178 km / s>

In [ ]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

In [12]:
scalar_EV = r_earth[0]*r_ven[0] + r_earth[1]*r_ven[1] +r_earth[2]*r_ven[2] 
scalar_VE_2 = r_earth_2[0]*r_ven[0] + r_earth_2[1]*r_ven[1] +r_earth_2[2]*r_ven[2]
scalar_E2J = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [13]:
fi_EV = (scalar_EV/norm(r_earth)/norm(r_ven))
fi_VE2 = (scalar_VE_2/norm(r_earth_2)/norm(r_ven))
fi_E2J = (scalar_E2J/norm(r_earth_2)/norm(r_jup))

In [14]:
np.degrees(np.arccos(fi_EV))

<Quantity 179.14218246 deg>

In [15]:
np.degrees(np.arccos(fi_VE2))

<Quantity 88.67695268 deg>

In [16]:
np.degrees(np.arccos(fi_E2J))

<Quantity 101.70296446 deg>

In [17]:
ist_an_1 = 2.6354471705049
ist_an_2 = 5.7770398240869

In [25]:
from math import*

exs = (norm(r_ven)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_ven)/norm(r_earth) * cos(ist_an_2)))
print(exs)

0.2008218331167458


In [26]:
p = norm(r_earth)*(1+cos(ist_an_1)*exs)
print(p)

125846316.317545 km


In [27]:
a = p/(1 - exs*exs)
print(a)

131134909.67911 km
